<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/figures/PDSH-cover-small.png?raw=1">

*This notebook contains an excerpt from the [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; the content is available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).*

*The text is released under the [CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode), and code is released under the [MIT license](https://opensource.org/licenses/MIT). If you find this content useful, please consider supporting the work by [buying the book](http://shop.oreilly.com/product/0636920034919.do)!*

<br>
<strong>Για το εργαστήριο του μαθήματος "<i>Στοχαστικές Διεργασίες & Βελτιστοποίηση στη Μηχανική Μάθηση</i>" έχουμε προσαρμόσει το κείμενο και τον κώδικα από το παραπάνω βιβλίο στις ανάγκες μας.</strong>

# Δέντρα Αποφάσεων (Decision Trees) και Random Forests

Τα Random Forests είναι ένα παράδειγμα ενός συνόλου τεχνικών μάθησης που βασίζεται σε δέντρα αποφάσεων. 

Τα δέντρα αποφάσεων αποτελούν έναν εξαιρετικά διαισθητικό τρόπο ταξινόμησης ή επισήμανσης αντικειμένων: απλώς υποβάλετε μια σειρά ερωτήσεων που έχουν σχεδιαστεί για να οδηγήσουν στην τελική ταξινόμηση. Για παράδειγμα, εάν θέλετε να δημιουργήσετε ένα δέντρο αποφάσεων για να ταξινομήσετε ένα ζώο που συναντάτε ενώ περπατάτε, μπορείτε να δημιουργήσετε αυτό που φαίνεται εδώ:

![alt text](https://jakevdp.github.io/PythonDataScienceHandbook/figures/05.08-decision-tree.png)

Ο δυαδικός διαχωρισμός είναι εξαιρετικά αποτελεσματικός: σε ένα καλά κατασκευασμένο δέντρο, κάθε ερώτηση θα μειώσει τον αριθμό των επιλογών κατά περίπου το ήμισυ, μειώνοντας πολύ γρήγορα τις επιλογές ακόμη και σε μεγάλο αριθμό τάξεων.

Το σημαντικό βέβαια σε αυτή τη προσέγγιση είναι να αποφασιστούν ποιες ερωτήσεις πρέπει να υποβληθούν σε κάθε βήμα. Στις εφαρμογές μηχανικής μάθησης των δένδρων αποφάσεων, οι ερωτήσεις γενικά έχουν τη μορφή διαχωρισμών άξονα στα δεδομένα: δηλαδή, κάθε κόμβος στο δέντρο χωρίζει τα δεδομένα σε δύο ομάδες χρησιμοποιώντας μια τιμή αποκοπής σε ένα από τα χαρακτηριστικά.

# Δημιουργία ενός Δέντρου Αποφάσεων


Έστω ότι έχουμε τα ακόλουθα δισδιάστατα δεδομένα, τα οποία είναι ταξινομημένα σε τέσσερις κλάσεις:

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [ ]:
from sklearn.datasets import make_blobs

X, y = make_blobs(n_samples=300, centers=4,
                  random_state=0, cluster_std=1.0)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='rainbow');

Ένα απλό δέντρο αποφάσεων που βασίζεται σε αυτά τα δεδομένα θα διαχωρίσει επαναληπτικά τα δεδομένα κατά μήκος του ενός ή του άλλου άξονα σύμφωνα με κάποιο ποσοτικό κριτήριο, και σε κάθε επίπεδο εκχωρεί την ετικέτα της νέας περιοχής σύμφωνα με την πλειοψηφία των σημείων μέσα σε αυτήν. Παρακάτω παρουσίαζεται μια απεικόνιση των πρώτων τεσσάρων επιπέδων ενός ταξινομητή δέντρων αποφάσεων για τα παραπάνω δεδομένα:

![alt text](https://jakevdp.github.io/PythonDataScienceHandbook/figures/05.08-decision-tree-levels.png)


Παρατηρούμε ότι μετά τον πρώτο διαχωρισμό, κάθε σημείο στον άνω κλάδο παραμένει αμετάβλητο, επομένως δεν χρειάζεται να υποδιαιρεθεί περαιτέρω αυτός ο κλάδος. Εκτός από τους κόμβους που περιέχουν ένα και μόνο χρώμα, σε κάθε επίπεδο κάθε περιοχή χωρίζεται ξανά σε ένα από τα δύο χαρακτηριστικά.


Αυτή η διαδικασία προσαρμογής ενός δέντρου αποφάσεων στα δεδομένα μας μπορεί να γίνει στο Scikit-Learn με τον εκτιμητή DecisionTreeClassifier:

In [3]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier().fit(X, y)

Παρακάτω παρατίθεται μία συνάρτηση χρησιμότητας για να απεικονίσουμε την έξοδο του ταξινομητή:

In [5]:
def visualize_classifier(model, X, y, ax=None, cmap='rainbow'):
    ax = ax or plt.gca()
    
    # Plot the training points
    ax.scatter(X[:, 0], X[:, 1], c=y, s=30, cmap=cmap,
               clim=(y.min(), y.max()), zorder=3)
    ax.axis('tight')
    ax.axis('off')
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # fit the estimator
    model.fit(X, y)
    xx, yy = np.meshgrid(np.linspace(*xlim, num=200),
                         np.linspace(*ylim, num=200))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

    # Create a color plot with the results
    n_classes = len(np.unique(y))
    contours = ax.contourf(xx, yy, Z, alpha=0.3,
                           levels=np.arange(n_classes + 1) - 0.5,
                           cmap=cmap, clim=(y.min(), y.max()),
                           zorder=1)

    ax.set(xlim=xlim, ylim=ylim)

In [ ]:
visualize_classifier(DecisionTreeClassifier(), X, y)

Παρακάτω παρατίθεται μία διαδραστική απεικόνιση της διαδικασίας δημιουργίας δέντρων αποφάσεων:

![alt text](...%0ASUVORK5CYII=)

Αυτό που προκύπτει είναι ότι καθώς αυξάνεται το βάθος του δέντρου, έχουμε την τάση να έχουμε πολύ παράξενα διαμορφωμένες περιοχές ταξινόμησης. Για παράδειγμα, σε βάθος πέντε, υπάρχει μια μικρή μωβ περιοχή μεταξύ των κίτρινων και των μπλε περιοχών. Αυτό είναι λιγότερο αποτέλεσμα της πραγματικής, εγγενούς διανομής δεδομένων και περισσότερο αποτέλεσμα των συγκεκριμένων ιδιοτήτων δειγματοληψίας ή θορύβου των δεδομένων.

Στο παραπάνω τμήμα κώδικα, ως κριτήριο επιλογής της σειράς των features στους κόμβους του Δέντρου Απόφασης χρησιμοποιήθηκε το Gini Index. 
<ul>
<li><b>Ερώτηση 1:</b> Να περιγράψετε πώς πραγματοποιείται η κατασκευή ενός Decision Tree με τη βοήθεια του Gini Index.</li>
</ul>

Επιπρόσθετα, ένα άλλο κριτήριο που χρησιμοποιείται για την κατασκευή Δέντρων Αποφάσεων είναι το Entropy.
<ul>
<li><b>Ερώτηση 2:</b> Να αναφέρετε τις διαφορές ανάμεσα στο Gini Index και στο κριτήριο Entropy.</li>
<li><b>Ερώτηση 3:</b> Ποια είναι τα πλεονεκτήματα του Gini Index έναντι του κριτηρίου Entropy;</li>
</ul>

## Δέντρα Αποφάσεων και over-fitting

Η υπερβολική προσαρμογή που φάνηκε παραπάνω, φαίνεται ότι αποτελεί γενική ιδιότητα των δέντρων αποφάσεων: είναι εύκολο με την αύξηση του βάθους του δέντρου να εξετάζονται περισσότερο οι λεπτομέρειες που έχει κάθε ζέυγος δεδομένων, παρά οι ιδιότητες των κατανομών από τις οποίες προέρχονται. Ένας αλλος τρόπος για να φανεί το over-fitting είναι να εξεταστούν μοντέλα που έχουν εκπαιδευτεί σε διαφορετικά υποσύνολα των δεδομένων - για παράδειγμα, στο παρακάτω σχήμα εκπαιδεύονται δύο διαφορετικά δέντρα, το καθένα στα μισά από τα αρχικά δεδομένα:

![alt text](https://jakevdp.github.io/PythonDataScienceHandbook/figures/05.08-decision-tree-overfitting.png)

Είναι σαφές ότι σε ορισμένα μέρη, τα δύο δέντρα παράγουν κοινά αποτελέσματα (π.χ. στις τέσσερις γωνίες), ενώ σε άλλα μέρη, τα δύο δέντρα δίνουν πολύ διαφορετικές ταξινομήσεις (π.χ. στις περιοχές μεταξύ οποιονδήποτε δύο συστάδων). Η βασική παρατήρηση είναι ότι οι ασυνέπειες τείνουν να συμβαίνουν όταν η ταξινόμηση είναι λιγότερο βέβαιη και, συνεπώς, χρησιμοποιώντας πληροφορίες και από τα δύο αυτά δέντρα, μπορεί να καταλήξουμε σε ένα καλύτερο αποτέλεσμα!

Παρακάτω φαίνονται προσαρμογές των δέντρων που έχουν εκπαιδευτεί σε ένα τυχαίο υποσύνολο των δεδομένων:

![alt text](...)

Παρατηρούμε ότι η χρήση πληροφοριών από δύο δέντρα βελτιώνει τα αποτελέσματά μας. Ως εκ τούτου η χρήση πληροφοριών από πολλά δέντρα θα βελτιώσει ακόμη περισσότερο τα αποτελέσματά μας.

Μία συνήθης μέθοδος για την αποφυγή του overfitting στα Δέντρα Αποφάσεων είναι το pruning. 
<ul>
<li><b>Ερώτηση 4:</b> Να περιγράψετε, σύντομα, το pruning των Δέντρων Αποφάσεων και να αναφέρετε πώς αυτό μπορεί να βοηθήσει στην αποφυγή του overfitting.</li>
</ul>

# Ταξινόμηση με τη χρήση Random Forests


Με τη χρήση Random Forests επιτυγχάνουμε τη χρήση παράλληλων εκτιμητών, καθένας από τους οποίους κάνει over-fit στα δεδομένα και υπολογίζει τα αποτελέσματα για να βρει μια καλύτερη ταξινόμηση.

Αυτός ο τύπος ταξινόμησης μπορεί να γίνει χειροκίνητα χρησιμοποιώντας τον μετα-εκτιμητή Scikit-Learn's BaggingClassifier, όπως φαίνεται εδώ:

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

tree = DecisionTreeClassifier()
bag = BaggingClassifier(tree, n_estimators=100, max_samples=0.8,
                        random_state=1)

bag.fit(X, y)
visualize_classifier(bag, X, y)


Σε αυτό το παράδειγμα, τα δεδομένα έχουν τυχαιοποιηθεί προσαρμόζοντας κάθε εκτιμητή με ένα τυχαίο υποσύνολο του 80% των σημείων εκπαίδευσης. Στην πράξη, τα δέντρα αποφάσεων τυχαιοποιούνται πιο αποτελεσματικά με τη στοχαστικότητα στον τρόπο επιλογής των διαχωρισμών: με αυτόν τον τρόπο όλα τα δεδομένα συμβάλλουν στην προσαρμογή κάθε φορά, αλλά τα αποτελέσματα της προσαρμογής εξακολουθούν να έχουν την επιθυμητή τυχαιότητα. Για παράδειγμα, κατά τον προσδιορισμό του χαρακτηριστικού στο οποίο θα χωριστεί, το τυχαιοποιημένο δέντρο μπορεί να επιλέξει μεταξύ των κορυφαίων λειτουργιών.

Στο Scikit-Learn, ένα τέτοιο βελτιστοποιημένο σύνολο τυχαιοποιημένων δέντρων αποφάσεων εφαρμόζεται στον εκτιμητή RandomForestClassifier, ο οποίος φροντίζει αυτόματα όλη την τυχαιοποίηση. Το μόνο που χρειάζεται είναι να επιλεγεί ένας αριθμός εκτιμητών ο οποίος και θα ταιριάξει πολύ γρήγορα (παράλληλα, αν θέλετε) στο σύνολο των δέντρων:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=0)
visualize_classifier(model, X, y);

<ul>
<li><b>Ερώτηση 5:</b> Δοκιμάστε να μειώσετε και να αυξήσετε τον αριθμό των εκτιμητών στις παραπάνω περιπτώσεις. Τι παρατηρείτε ως προς την ακρίβεια και το χρόνο εκτέλεσης της μεθόδου Random Forests;</li>
<li><b>Ερώτηση 6:</b> Χρειάζεται η εφαρμογή του pruning στην περίπτωση των Random Forests, όπως στην περίπτωση των Δέντρων Αποφάσεων; Να δικαιολογήσετε την απάντησή σας.</li>
</ul>

# Παλινδρόμηση με Random Forest


Τα Random Forests δεν χρησιμοποιούνται μόνο στο πλαίσιο της ταξινόμησης στοιχείων, αλλά μπορούν επίσης να χρησιμοποιηθούν και σε περίπτωση παλινδρόμησης. Ο εκτιμητής που θα χρησιμοποιηθεί για αυτό είναι το RandomForestRegressor και η σύνταξη είναι παρόμοια με τη περίπτωση της ταξινόμησης.

Έστω ότι έχουμε τα ακόλουθα δεδομένα, που αντλούνται από το συνδυασμό γρήγορης και αργής ταλάντωσης:

In [ ]:
rng = np.random.RandomState(42)
x = 10 * rng.rand(200)

def model(x, sigma=0.3):
    fast_oscillation = np.sin(5 * x)
    slow_oscillation = np.sin(0.5 * x)
    noise = sigma * rng.randn(len(x))

    return slow_oscillation + fast_oscillation + noise

y = model(x)
plt.errorbar(x, y, 0.3, fmt='o');


Χρησιμοποιώντας παλινδρόμηση βασισμένη στα Random Forests, μπορούμε να βρούμε τη καμπύλη βελτιστοποίησης ως εξής:

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(200)
forest.fit(x[:, None], y)

xfit = np.linspace(0, 10, 1000)
yfit = forest.predict(xfit[:, None])
ytrue = model(xfit, sigma=0)

plt.errorbar(x, y, 0.3, fmt='o', alpha=0.5)
plt.plot(xfit, yfit, '-r');
plt.plot(xfit, ytrue, '-k', alpha=0.5);

Εδώ το αληθινό μοντέλο εμφανίζεται στην γκρι καμπύλη, ενώ το μοντέλο που δημιουργήθηκε με Random Forests εμφανίζεται με την οδοντωτή κόκκινη καμπύλη. Όπως φαίνεται, το μη παραμετρικό μοντέλο βασισμένο σε Random Forests είναι αρκετά ευέλικτο ώστε να χωράει τα δεδομένα πολλαπλών περιόδων, χωρίς να χρειάζεται να προσδιορίσουμε ένα μοντέλο πολλαπλών περιόδων!

# Παράδειγμα: Random Forest για κατηγοριοποίηση ψηφίων

Εισάγουμε τις απαραίτητες βιβλιοθήκες και το σχετικό dataset με τα χειρόγραφα ψηφία

In [11]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [12]:
from sklearn.datasets import load_digits
digits = load_digits()
digits.keys()

dict_keys(['data', 'target', 'frame', 'feature_names', 'target_names', 'images', 'DESCR'])

Οπτικοποίηση μερικών ψηφίων

In [ ]:
# set up the figure
fig = plt.figure(figsize=(6, 6))  # figure size in inches
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

# plot the digits: each image is 8x8 pixels
for i in range(64):
    ax = fig.add_subplot(8, 8, i + 1, xticks=[], yticks=[])
    ax.imshow(digits.images[i], cmap=plt.cm.binary, interpolation='nearest')
    
    # label the image with the target value
    ax.text(0, 7, str(digits.target[i]))

Κατηγοριοποιούμε τα ψηφία με Random Forest

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

Xtrain, Xtest, ytrain, ytest = train_test_split(digits.data, digits.target,
                                                random_state=0)
model = RandomForestClassifier(n_estimators=1000)
model.fit(Xtrain, ytrain)
ypred = model.predict(Xtest)

In [ ]:
from sklearn import metrics
print(metrics.classification_report(ypred, ytest))

Εμφάνιση των λαθών μέσω της [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix)

In [ ]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(ytest, ypred)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('true label')
plt.ylabel('predicted label');

<ul>
<li><b>Ερώτηση 7:</b> Να αναφέρετε για ποιους λόγους απαιτούνται οι μετρικές precision, recall και f1-score πέραν της ακρίβειας (accuracy).</li>
<li><b>Ερώτηση 8:</b> Ποιες άλλες μέθοδοι χρησιμοποιούνται, συνήθως, για την αξιολόγηση μοντέλων μηχανικής μάθησης; Να αναφέρετε μερικές και να αναφέρετε τα πλεονεκτήματά τους (hint: <a href="https://towardsdatascience.com/various-ways-to-evaluate-a-machine-learning-models-performance-230449055f15">Μέθοδοι)</a></li>
</ul>